In [1]:
from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()
search.invoke("what is the weather in SF")

[{'url': 'https://www.weatherapi.com/',
  'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737260885, 'localtime': '2025-01-18 20:28'}, 'current': {'last_updated_epoch': 1737260100, 'last_updated': '2025-01-18 20:15', 'temp_c': 10.2, 'temp_f': 50.4, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.3, 'wind_kph': 10.1, 'wind_degree': 300, 'wind_dir': 'WNW', 'pressure_mb': 1025.0, 'pressure_in': 30.26, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 83, 'cloud': 0, 'feelslike_c': 8.9, 'feelslike_f': 47.9, 'windchill_c': 7.8, 'windchill_f': 46.0, 'heatindex_c': 8.6, 'heatindex_f': 47.5, 'dewpoint_c': 7.9, 'dewpoint_f': 46.2, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 10.8, 'gust_kph': 17.4}}"},
 {'url': 'https://weathershogun.com/weather/usa/ca/

## Creating Retrieval Tool

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)

embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector = FAISS.from_documents(documents, embedding )
retriever = vector.as_retriever()

C:\Users\mayur\AppData\Local\Temp\ipykernel_11576\3029265589.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\mayur\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
retriever.invoke("how to upload a dataset")[0]

Document(id='395444a5-3c03-4e96-a899-c84f14c16305', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Run the evaluationexperiment_results = client.evaluate(    dummy_app, # Your AI system goes here    data=dataset, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={"version": "1.0.0", "revision_id": "beta"}, # Metadata about the experiment    max_concurrency=4,  # Add concurrency.)# Analyze the results via the UI or programmatically# If you have \'pandas\' installed you can view the results as a# pandas DataFrame by uncommenting below:# experiment_results.to_pandas()import { Client } from "langsmith";import { EvaluationResult, evaluate } from "langsmith/evalua

In [5]:
from langchain.tools.retriever import create_retriever_tool

In [6]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

## Tools

In [7]:
tools = [search, retriever_tool]

## Create the Agent

In [10]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="Llama3-8b-8192")


In [17]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [23]:
# from langchain import 
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """You are a helpful assistant. Your task is to ckeck if input contain word - "langchain" then use retrieval tool otherwise use web search tool"""),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [24]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [25]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [28]:
agent_executor.invoke({"input": "what is langsmith"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'what is langsmith'}`


[{'url': 'https://www.geeksforgeeks.org/introduction-to-langsmith/', 'content': 'Langsmith is an innovative framework designed to enhance and streamline the development of natural language processing (NLP) applications. It builds upon LangChain, a popular library for chaining multiple language models together, to create sophisticated and flexible NLP workflows. Langsmith provides tools for managing, deploying, and scaling'}, {'url': 'https://www.langchain.ca/blog/what-is-langsmith/', 'content': 'Langsmith is an application framework for language models equipped with LLM tools and agent support. It focuses on addressing challenges faced during language model development for production. This guide offers an in-depth look into the essential features and functionalities of Langsmith. Understanding Langsmith vs Langchain Prior to exploring Langsmith, it'}, {'url': 'https:/

{'input': 'what is langsmith',
 'output': 'Based on the output of the tool call, it appears that Langsmith is an application framework for language models that provides tools for managing, deploying, and scaling language models. It also seems to be designed to simplify the development of natural language processing (NLP) applications by providing a robust DevOps platform for large language models.\n\nSome key features of Langsmith mentioned in the output include:\n\n* Log management: Langsmith logs all calls to language models, chains, agents, tools, and retrievers, which can be helpful for debugging and troubleshooting.\n* Chain management: Langsmith allows users to create and manage chains of language models, which can be useful for building complex NLP workflows.\n* Agent support: Langsmith provides support for agents, which can be used to automate tasks and workflows.\n* LLM tools: Langsmith provides tools for working with large language models (LLMs), including tools for debugging